In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('CV/CV10/cross_validation.csv', sep=';')

In [ ]:
def select_rows(df: pd.DataFrame, model: str) -> pd.DataFrame:
    if model in ('logreg', 'rf'):
        bias_mitigator = lambda frame: frame['Bias Mitigator'].isnull()
    else:
        bias_mitigator = lambda frame: frame['Bias Mitigator'] == ('Reweighting' if model.endswith('reweight') else 'Prejudice Remover')
    clf = 'Random Forest' if model.startswith('rf') else 'Logistic Regression'
    return df[(bias_mitigator(df)) & (df['Classifier'] == clf)]

def get_mean_stdev(model: str) -> pd.DataFrame:
    selection = select_rows(df, model)
    means = selection.mean()
    stds = selection.std()
    mean_std = pd.concat([means, stds], axis=1).rename(columns={0: 'Mean', 1: 'StdDev'})
    mean_std[model] = mean_std.apply(lambda row: ' +/- '.join(['{:.3f}'.format(row[el]) for el in ('Mean', 'StdDev')]), 1)
    report = mean_std.drop(columns=['Mean', 'StdDev']).transpose()
    report.drop(columns=[el for el in ('Bias Mitigator', 'fold') if el in report.columns], inplace=True)
    return report

### Logistic Regression

In [ ]:
get_mean_stdev('logreg')

### Logistic Regression with Reweighting

In [ ]:
get_mean_stdev('logregreweight')

### Random Forest

In [ ]:
get_mean_stdev('rf')

### Random Forest with Reweighting

In [ ]:
get_mean_stdev('rfreweight')

### Logistic Regression with Prejudice Remover

In [ ]:
get_mean_stdev('prejudiceremover')

### All together

In [ ]:
pd.concat([get_mean_stdev(el) for el in ('logreg', 'rf', 'logregreweight', 'rfreweight', 'prejudiceremover')])